#### <font color='maroon'><b>Amazon - Microwave oven - Web Scraping - using Selenium :

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urljoin
import re
import pandas as pd
import numpy as np
import copy

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Configure Chrome options to disable images and ads
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--dns-prefetch-disable")

In [3]:
# Set up Chrome WebDriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

In [4]:
# Base URL for the search result page
base_url = "https://www.amazon.in/s?k=microwave+oven&page={}&qid=1687332637&ref=sr_pg_1"

# Scrape product links
product_links = set()

# Start from page 1
page_number = 1

while True:
    # Construct the URL for the current page
    url = base_url.format(page_number)

    # Open the current page
    driver.get(url)

    # Wait for the page to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-component-type='s-search-result']")))

    # Find all product links on the current page
    links = driver.find_elements(By.CSS_SELECTOR, "a.a-link-normal.a-text-normal")
    for link in links:
        href = link.get_attribute("href")
        if "/dp/" in href:
            product_links.add(href)

    # Check if there is a next page
    next_button = driver.find_elements(By.CSS_SELECTOR, "a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator")
    if len(next_button) == 0:
        break

    # Increment the page number
    page_number += 1

# Print all the product links
for link in product_links:
    print(link)

# Close the WebDriver
#driver.quit()


https://aax-eu.amazon.in/x/c/RDJEAB3aDVDmxJ_vZ7DYMF0AAAGI_BfknQMAAAH2AQBvbm9fdHhuX2JpZDIgICBvbm9fdHhuX2ltcDEgICCzh4dz/https://www.amazon.in/Tesora-Replaces-Rotisserie-Circulation-Technology/dp/B09VGTZYTZ/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c%3Aamzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c&cv_ct_cx=microwave+oven&keywords=microwave+oven&pd_rd_i=B09VGTZYTZ&pd_rd_r=e5c00fcb-e30a-45c6-981c-a19f34df80fb&pd_rd_w=giVfb&pd_rd_wg=mcTd7&pf_rd_p=f94148a7-b034-4e58-9f1d-2cf8db3cd03c&pf_rd_r=GEJPEFFYQN55K6VBER81&qid=1687856604&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-113-06195d41-65bd-48f4-8e70-03d97b6094ec
https://www.amazon.in/Wonderchef-Roland-63152505-Litre-Microwave/dp/B07KMLTT7T/ref=sr_1_101?keywords=microwave+oven&qid=1687856601&sr=8-101
https://www.amazon.in/Faber-Convection-FBIO-80L-6F/dp/B0821ZJR98/ref=sr_1_82?keywords=microwave+oven&qid=1687856598&sr=8-82
https://www.amazon.in/7CF-700-Microwave-BkSS-OB/dp/B008S792R4/ref=sr_1_115?keywords=mi

In [5]:
# number of scraped products

len(product_links)

238

In [6]:
product_title = []
sales_price = []
mrp = []
discounts = []
ratings = []
no_of_ratings = []
seller_name = []
product_details =[]
technical_details = []
additional_info = []

In [7]:
for link in product_links:
    driver.get(link)
    driver.implicitly_wait(10)
    
    # titles
    
    try:
        title = driver.find_element(By.ID, "productTitle").text
        product_title.append(title)
    except:
        product_title.append('NA')
        
    
    # sales price

    try:
        sl_pr = driver.find_element(By.CSS_SELECTOR, "#newAccordionRow_0 span.a-color-price").text.strip()
        sales_price.append(sl_pr)
    except NoSuchElementException:
        try:
            sl_pr = driver.find_element(By.CSS_SELECTOR, "div.a-section.a-spacing-none.aok-align-center span.a-price").text.strip()
            sales_price.append(sl_pr)
        except NoSuchElementException:
            sales_price.append('NA')
   
    
    # market price
    
    try:
        mk_pr = driver.find_element(By.CSS_SELECTOR, "#newAccordionRow_0 span.a-color-secondary").text.strip()
        mrp.append(mk_pr)
    except NoSuchElementException:
        try:
            mk_pr = driver.find_element(By.CSS_SELECTOR, "span.a-price.a-text-price").text.strip()
            mrp.append(mk_pr)
        except NoSuchElementException:
            mrp.append('NA')
            
       
    # discounts
    
    try:
        discount = driver.find_element(By.CSS_SELECTOR, "div#corePriceDisplay_desktop_feature_div.celwidget span").text
        discounts.append(discount)
    except NoSuchElementException:
        try:
            discount = driver.find_element(By.CSS_SELECTOR, "div.a-section.a-spacing-none.aok-align-center span.a-size-large").text
            discounts.append(discount)
        except NoSuchElementException:
            discounts.append('NA')
                
    
    # product ratings
    
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, "div#averageCustomerReviews span.a-icon-alt")
        rating = rating_element.get_attribute("innerHTML").split()[0]
        ratings.append(rating)
    except NoSuchElementException:
        try:
            rating = driver.find_element(By.CSS_SELECTOR, "span#acrPopover").text
            ratings.append(rating)
        except NoSuchElementException:
            ratings.append('NA')
                
            
    # number of ratings
    
    try:
        no = driver.find_element(By.CSS_SELECTOR, "#acrCustomerReviewText").text.split()[0]
        no_of_ratings.append(no)
    except NoSuchElementException:
        try:
            no = driver.find_element(By.CSS_SELECTOR, "span#acrCustomerReviewText").text
            no_of_ratings.append(no)
        except NoSuchElementException:
            no_of_ratings.append('NA')
    
    
   # seller 
    
    try:
        sell = driver.find_element(By.CSS_SELECTOR, "#merchant-info a").text
        seller_name.append(sell)
    except NoSuchElementException:
        try:
            sell = driver.find_element(By.CSS_SELECTOR, "div#merchant-info a.a-link-normal").text
            seller_name.append(sell)
        except NoSuchElementException:
            seller_name.append('NA')
          
        
    # product details
        
    try:
        table = driver.find_element(By.CSS_SELECTOR, "table.a-normal.a-spacing-micro")

        # Initialize a dictionary to store the scraped data
        microwave_details = {}

        # Iterate through each row in the table
        rows = table.find_elements(By.TAG_NAME, "tr")
        for row in rows:
            # Get the cells in the row
            cells = row.find_elements(By.TAG_NAME, "td")
    
            # Extract the brand and capacity values
            if len(cells) >= 2:
                brand = cells[0].text.strip()
                capacity = cells[1].text.strip()
        
                # Store the brand and capacity as key-value pairs in the dictionary
                microwave_details[brand] = capacity
        product_details.append(microwave_details)
    except:
        product_details.append({})
        
            
    # technical details
    
    try:
        tech_details_table = driver.find_element(By.CSS_SELECTOR, "#productDetails_techSpec_section_1")
        rows = tech_details_table.find_elements(By.XPATH, ".//tr")

        details = {}
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th|td")
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                details[key] = value

        technical_details.append(details)
    except:
        technical_details.append({})
        
        
    # additional information
        
    try:
        adl_details_table = driver.find_element(By.CSS_SELECTOR, "#productDetails_db_sections")
        rows = adl_details_table.find_elements(By.XPATH, ".//tr")

        details = {}
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th|td")
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                details[key] = value

        additional_info.append(details)
    except:
        additional_info.append({})
            

In [8]:
len(product_title)

238

In [10]:
print('product_title')
print()
print(product_title)
print('-'*50)
print('sales_price')
print()
print(sales_price)
print('-'*50)
print('market retail price')
print()
print(mrp)
print('-'*50)
print('discounts')
print()
print(discounts)
print('-'*50)
print('star_rating')
print()
print(ratings)
print('-'*50)
print('no_of_ratings')
print()
print(no_of_ratings)
print('-'*50)
print('seller_name')
print()
print(seller_name)
print('-'*50)
print()
print(product_details)
print('-'*50)
print('technical_details')
print()
print(technical_details)
print('-'*50)
print('additional_info')
print()
print(additional_info)

product_title

['Tesora Digital Air Fryer Oven | Replaces OTG, Oven, Air Fryer, Toaster, Rotisserie, OTG | 360° Turbo Hot Air Circulation Technology & Dual Heating Element| 1700 Watts | Black.(14.5L) (14.5L)', 'Wonderchef Roland 30 Litre Microwave (Black)', 'Faber 80 L Convection Microwave Oven (FBIO 80L 6F, Black)', '.7CF 700 W Microwave BkSS OB', 'Essencedelight Microwave Oven Cover Dustproof Cotton Machine Protector Decorative Kitchen Appliance Cover with Side Storage Pockets,lattice', "H'A'FELE Built-In Microwave Oven With Grill AIDA 28 (Black,25 Litres)", 'LG 32 L Convection Microwave Oven (MC3286BIUM, Black, Heart Friendly Recipes)', 'Usha CALYPSO Digital Air Fryer Oven Toaster Grill (OTG) 30 Litre capacity All in one- Air Fry, Bake, Dehydrate, Toast, Broil | Digital Display | 8 Cooking Options, 8 accessories |2 Yrs Warranty Black', 'Bosch Oven Slide 658253 00658253', 'Countertop 1.1 Cubic Feet Microwave Oven, 1000 Watt, Black Front with Black Cabinet, Commercial Chef CHCM11100B'

#### <font color='maroon'><b>Data Cleaning :

In [21]:
# convert set to list
prod_links_list = list(product_links)

In [49]:
# pandas dataframe
df1 = pd.DataFrame({'product_url':prod_links_list,
                  'product_title':product_title,
                  'mrp':mrp,
                  'sales_price':sales_price,
                  'discount_percent':discounts,
                  'star_rating':ratings,
                  'number_of_ratings':no_of_ratings,
                  'seller_name':seller_name,
                  'product_details':product_details,
                  'technical_details':technical_details,
                  'additional_information':additional_info})

In [50]:
# check for duplicates

df1.duplicated(subset='product_title').sum()

28

In [51]:
# drop duplicates

df1.drop_duplicates(subset=['product_title'],keep='first',inplace=True)

In [42]:
df1.shape

(210, 11)

In [24]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth',None)

In [58]:
# checking for incorrect values from scraping

df1[['product_url','discount_percent','sales_price','mrp']]

,product_url,discount_percent,sales_price,mrp
0,https://aax-eu.amazon.in/x/c/RDJEAB3aDVDmxJ_vZ7DYMF0AAAGI_BfknQMAAAH2AQBvbm9fdHhuX2JpZDIgICBvbm9fdHhuX2ltcDEgICCzh4dz/https://www.amazon.in/Tesora-Replaces-Rotisserie-Circulation-Technology/dp/B09VGTZYTZ/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c%3Aamzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c&cv_ct_cx=microwave+oven&keywords=microwave+oven&pd_rd_i=B09VGTZYTZ&pd_rd_r=e5c00fcb-e30a-45c6-981c-a19f34df80fb&pd_rd_w=giVfb&pd_rd_wg=mcTd7&pf_rd_p=f94148a7-b034-4e58-9f1d-2cf8db3cd03c&pf_rd_r=GEJPEFFYQN55K6VBER81&qid=1687856604&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-113-06195d41-65bd-48f4-8e70-03d97b6094ec,-30%,"₹12,999","₹18,500"
1,https://www.amazon.in/Wonderchef-Roland-63152505-Litre-Microwave/dp/B07KMLTT7T/ref=sr_1_101?keywords=microwave+oven&qid=1687856601&sr=8-101,-24%,"18,999.00","25,000.00"
2,https://www.amazon.in/Faber-Convection-FBIO-80L-6F/dp/B0821ZJR98/ref=sr_1_82?keywords=microwave+oven&qid=1687856598&sr=8-82,-7%,"39,190.00","41,990.00"
3,https://www.amazon.in/7CF-700-Microwave-BkSS-OB/dp/B008S792R4/ref=sr_1_115?keywords=microwave+oven&qid=1687856604&sr=8-115,"₹26,708\n89","₹26,708\n89","₹15,600.00"
4,https://www.amazon.in/Essencedelight-Microwave-Dustproof-Protector-Decorative/dp/B0897SXY7X/ref=sr_1_197?keywords=microwave+oven&qid=1687856619&sr=8-197,-50%,"₹2,089","₹4,199"
5,https://www.amazon.in/HAFELE-Built-Microwave-AIDA-28/dp/B0BQ6ZQ8BC/ref=sr_1_179?keywords=microwave+oven&qid=1687856616&sr=8-179,NA,NA,NA
6,https://www.amazon.in/LG-Convection-Microwave-MC3286BIUM-Friendly/dp/B0C4FHT3QK/ref=sr_1_106?keywords=microwave+oven&qid=1687856601&sr=8-106,-9%,"22,751.00","24,999.00"
7,https://aax-eu.amazon.in/x/c/RNs86FhdQbKKhv8-4-yvdEwAAAGI_Bgf8QMAAAH2AQBvbm9fdHhuX2JpZDEgICBvbm9fdHhuX2ltcDEgICB1yje6/https://www.amazon.in/dp/B09CH7956L/?_encoding=UTF8&pd_rd_i=B09CH7956L&pd_rd_w=yRD64&content-id=amzn1.sym.69a9d8c7-1563-43e8-bc18-abd50d05a73a%3Aamzn1.sym.69a9d8c7-1563-43e8-bc18-abd50d05a73a&pf_rd_p=69a9d8c7-1563-43e8-bc18-abd50d05a73a&pf_rd_r=NJF86EAPGKWW39BYJZRV&pd_rd_wg=ObhWK&pd_rd_r=ef1ef945-e6bf-447d-80d0-01dc99996544&ref_=sbv_search_btf&&pd_rd_plhdr=t,-23%,"₹12,349","₹15,990"
8,https://www.amazon.in/Bosch-Oven-Slide-658253-00658253/dp/B00LKOSDUY/ref=sr_1_203?keywords=microwave+oven&qid=1687856619&sr=8-203,"₹33,562\n75","₹33,562\n75","₹15,600.00"
9,https://www.amazon.in/Westinghouse-WCM11100B-Counter-Microwave-Cabinet/dp/B00GEC80EG/ref=sr_1_110?keywords=microwave+oven&qid=1687856601&sr=8-110,-30%,"₹29,193","₹41,709"


In [66]:
# correcting the errors

df1.at[3,'mrp'] = 'NA'
df1.at[3,'sales_price'] = 26708
df1.at[3,'discount_percent'] = 'NA'

In [59]:
df1.at[237,'discount_percent'] ='NA'

In [60]:
df1.at[206,'discount_percent'] ='NA'

In [61]:
df1.at[179,'mrp'] = 'NA'
df1.at[179,'sales_price'] = 76569
df1.at[179,'discount_percent'] = 'NA'

In [62]:
df1.at[170,'mrp'] = 'NA'
df1.at[170,'sales_price'] = 57990
df1.at[170,'discount_percent'] = 'NA'

In [63]:
df1.at[166,'mrp'] = 'NA'
df1.at[166,'sales_price'] = 'NA'
df1.at[166,'discount_percent'] = 'NA'

In [64]:
df1.at[126,'mrp'] = 'NA'
df1.at[126,'sales_price'] = 35266
df1.at[126,'discount_percent'] = 'NA'

In [65]:
df1.at[120,'mrp'] = 'NA'
df1.at[120,'sales_price'] = 129900
df1.at[120,'discount_percent'] = 'NA'

In [67]:
df1.at[117,'mrp'] = 'NA'
df1.at[117,'sales_price'] = 51005
df1.at[117,'discount_percent'] = 'NA'

In [68]:
df1.at[116,'mrp'] = 'NA'
df1.at[116,'sales_price'] = 102490
df1.at[116,'discount_percent'] = 'NA'

In [69]:
df1.at[113,'mrp'] = 'NA'
df1.at[113,'sales_price'] = 30500
df1.at[113,'discount_percent'] = 'NA'

In [70]:
df1.at[101,'discount_percent'] ='NA'

In [71]:
df1.at[67,'mrp'] = 'NA'
df1.at[67,'discount_percent'] = 'NA'

In [72]:
df1.at[8,'mrp'] = 'NA'
df1.at[8,'sales_price'] = 33562
df1.at[8,'discount_percent'] = 'NA'

In [73]:
df1.at[15,'discount_percent'] ='NA'

In [74]:
df1.at[46,'mrp'] = 14799
df1.at[46,'sales_price'] = 14047
df1.at[46,'discount_percent'] = 5

In [76]:
df2 = copy.deepcopy(df1)

In [78]:
# removing the special characters from values

def remove_special_chars(df, column_names):
    for column_name in column_names:
        df[column_name] = df[column_name].str.replace('₹', '').str.replace('%', '').str.replace('-', '').str.replace(',', '')
    return df

In [79]:
df2 = remove_special_chars(df2, ['mrp', 'sales_price', 'discount_percent','number_of_ratings'])

In [80]:
df2[['mrp', 'sales_price', 'discount_percent','number_of_ratings']]

,mrp,sales_price,discount_percent,number_of_ratings
0,18500,12999,30,117
1,25000.00,18999.00,24,16
2,41990.00,39190.00,7,37
3,NA,NaN,NA,40
4,4199,2089,50,NA
5,NA,NA,NA,NA
6,24999.00,22751.00,9,NA
7,15990,12349,23,109
8,NA,NaN,NA,NA
9,41709,29193,30,1452


In [81]:
df2[['mrp', 'sales_price', 'discount_percent','number_of_ratings','star_rating']].dtypes

mrp                  object
sales_price          object
discount_percent     object
number_of_ratings    object
star_rating          object
dtype: object

In [83]:
df2.replace('NA', np.nan, inplace=True)

In [84]:
df2['sales_price'] = df2['sales_price'].astype(float).astype('Int64')

In [85]:
df2['mrp'] = df2['mrp'].astype(float).astype('Int64')

In [86]:
df2['discount_percent'] = df2['discount_percent'].astype(float).astype('Int64')

In [87]:
df2['number_of_ratings'] = df2['number_of_ratings'].astype(float).astype('Int64')

In [88]:
df2['star_rating'] = df2['star_rating'].astype(float)

In [89]:
df2[['mrp', 'sales_price', 'discount_percent','number_of_ratings','star_rating']].dtypes

mrp                    Int64
sales_price            Int64
discount_percent       Int64
number_of_ratings      Int64
star_rating          float64
dtype: object

In [184]:
# product details
dfp = pd.DataFrame(product_details)
dfp.head()

,Special Feature,Product Dimensions,Colour,Capacity,Material,Brand,Installation Type,Wattage,Control Type,Heating Method,Finish Type,Fuel Type,Recommended Uses For Product,Size,Timer Function,Included Components,Door Style,Model Name,Controller Type,Item Dimensions LxWxH,Defrost System
0,Temperature Control,30D x 30W x 30H Centimeters,Black,14.5 litres,Aluminium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Timer, Turntable",NaN,Black,30 litres,NaN,Wonderchef,Countertop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Timer, Programmable",59D x 56W x 59H Centimeters,Black,80 litres,NaN,Faber,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Turntable,NaN,NaN,0.7 Cubic Feet,NaN,Avanti,Countertop,700 Watts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
dfp[['Capacity','Size']]

,Capacity,Size
0,14.5 litres,NaN
1,30 litres,NaN
2,80 litres,NaN
3,0.7 Cubic Feet,NaN
4,NaN,NaN
5,25 litres,NaN
6,32 litres,NaN
7,NaN,NaN
8,NaN,NaN
9,1.1 Cubic Feet,NaN


In [185]:
dfp.at[23,'Capacity'] = 20
dfp.at[38,'Capacity'] = 21
dfp.at[43,'Capacity'] = 25
dfp.at[66,'Capacity'] = 28
dfp.at[75,'Capacity'] = 28
dfp.at[91,'Capacity'] = 60
dfp.at[102,'Capacity'] = 20
dfp.at[104,'Capacity'] = 17
dfp.at[144,'Capacity'] = 71
dfp.at[153,'Capacity'] = 28
dfp.at[157,'Capacity'] = 71
dfp.at[184,'Capacity'] = 28
dfp.at[204,'Capacity'] = 17
dfp.at[220,'Capacity'] = 23
dfp.at[221,'Capacity'] = 25

In [186]:
dfp.at[3,'Capacity'] = int(0.7*28.31)
dfp.at[9,'Capacity'] = int(1.1*28.31)
dfp.at[106,'Capacity'] = int(0.6*28.31)
dfp.at[126,'Capacity'] = int(0.7*28.31)

In [174]:
def extract_capacity(string):
    matches = re.findall(r'\d+\.?\d*', string)
    if matches:
        return matches[0]
    else:
        return np.nan

In [187]:
dfp['Capacity'] = dfp['Capacity'].astype(str).apply(extract_capacity)

In [188]:
dfp['Capacity'][:10]

0    14.5
1      30
2      80
3      19
4     NaN
5      25
6      32
7     NaN
8     NaN
9      31
Name: Capacity, dtype: object

In [189]:
# technical details in pandas dataframe

df_tech = pd.DataFrame(technical_details)
df_tech.head()

,Special Feature,Product Dimensions,Colour,Capacity,Material,Recommended Uses For Product,Output Wattage,Item Weight,Brand,Wattage,Voltage,Control Method,Has Nonstick Coating,Inner Material,Min Temperature Setting,Controller Type,Is Dishwasher Safe,Max Temperature Setting,Manufacturer,Country of Origin,Item model number,ASIN,Model,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Fuel Type,Defrost System,Door Orientation,Included Components,Batteries Required,Model Name,Model Year,Mounting Hardware,Colour Screen,Audio Wattage,Batteries Included,GSM frequencies,Device interface - primary,Does it contain liquid?,Includes Rechargeable Battery,Is there a timer?,Includes remote,Item part number,Energy Consumption,Form Factor,Control Type,Door Style,Finish Type,Door Material Type,Volume Capacity Name,Energy Efficiency,Lighting,Certification,Control Console,Imported By,Number of Memory Sticks,Special features,Other display features,Color,Item Model Number,Item Part Number,Primary material,What is in the box?,Shelf Type,Whats in the box,Is Assembly Required,Assembly Type,Number of Pieces,Number of Shelves,Locking Mechanism,Weight,Item Height,Item Width,Door Material,Chamber Volume,Number of Heating Elements,Power Source
0,Temperature Control,30D x 30W x 30H Centimeters,Black,14.5 litres,Aluminium,Indoor Kitchen,1700 Watts,10.5 Kilograms,Tesora - Inspired by you,1700 Watts,230 Volts,Touch,Yes,Stainless Steel,50 Degrees Celsius,Button,Yes,220 Degrees Celsius,Tesora-inspired by you,China,14.5L,B09VGTZYTZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wonderchef,China,Wonderchef Roland Microwave 30Litre,B07KMLTT7T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Black,80 litres,Steel,NaN,NaN,NaN,Faber,2000 Watt Hours,230 Volts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Faber,India,NaN,NaN,FBIO 80L 6F,2835 Watts,Built-In,116.0526.885,"Timer, Programmable",Convection,Ceramic,Electric,Defrost,Left,"1 Microwave Oven, 1 User Manual, 1 Warranty Card",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,32.39 x 45.09 x 26.04 cm; 10.39 Kilograms,NaN,NaN,NaN,NaN,NaN,10 kg 400 g,Avanti,700 Watts,115 Volts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Avanti,NaN,MO7103SST,NaN,MO7103SST,NaN,NaN,NaN,Turntable,NaN,NaN,NaN,NaN,NaN,NaN,No,MO7103SST,2013,Turntable,No,700 Watts,No,60 Hz,Touchscreen,No,No,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Essencedelight,China,NaN,B0897SXY7X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Essencedelight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [152]:
# checking for percentage of nan values

df_tech['Installation Type'].isna().mean()*100

34.03361344537815

In [153]:
dfp['Installation Type'].isna().mean()*100

70.58823529411765

In [156]:
dfp['Capacity'].isna().mean()*100

45.79831932773109

In [157]:
df_tech['Capacity'].isna().mean()*100

32.773109243697476

In [190]:
df_tech['Capacity'] = df_tech['Capacity'].astype(str).apply(extract_capacity)
df_tech['Capacity'][:10]

0    14.5
1     NaN
2      80
3     NaN
4     NaN
5      25
6      32
7     NaN
8     NaN
9     NaN
Name: Capacity, dtype: object

In [191]:
df_tech['Capacity'].fillna(dfp['Capacity'], inplace=True)
df_tech['Capacity']

0      14.5
1        30
2        80
3        19
4       NaN
5        25
6        32
7       NaN
8       NaN
9        31
10       24
11       32
12      NaN
13       20
14       27
15       20
16       66
17       25
18       29
19       25
20       34
21       20
22       20
23       20
24      NaN
25       70
26      NaN
27       20
28      NaN
29     14.5
30       28
31       65
32       17
33       24
34       20
35       30
36      NaN
37       20
38       21
39       30
40      NaN
41       25
42       28
43       25
44      NaN
45       25
46      NaN
47       25
48       28
49       65
50       30
51      NaN
52      NaN
53       28
54       20
55       23
56      NaN
57       42
58      NaN
59      NaN
60     14.5
61       29
62       19
63       67
64       29
65       28
66       28
67      NaN
68       28
69       28
70      NaN
71       23
72      NaN
73      NaN
74       28
75       28
76      NaN
77       20
78      NaN
79      NaN
80       20
81       28
82       32
83  

In [192]:
df_tech['Capacity'].isna().mean()*100

26.47058823529412

In [205]:
df_tech.at[106,'Capacity'] = int(0.6*28.31)

In [197]:
df_tech['Capacity'] = df_tech['Capacity'].str.strip()

In [201]:
# Define the pattern for matching 14.5 with potential formatting issues
pattern = r'^\s*14\.5\s*$'

# Replace values matching the pattern with '14'
df_tech['Capacity'] = df_tech['Capacity'].apply(lambda x: re.sub(pattern, '14', x))

In [202]:
df_tech['Capacity'][:5]

0     14
1     30
2     80
3     19
4    nan
Name: Capacity, dtype: object

In [206]:
df_tech['Capacity'] = df_tech['Capacity'].astype(float).astype('Int64')

In [158]:
dfp['Brand'].isna().mean()*100

10.92436974789916

In [159]:
df_tech['Brand'].isna().mean()*100

22.689075630252102

In [207]:
df_tech.drop(['ASIN','Item Weight'],axis=1,inplace=True)

In [208]:
df_tech.drop(['Brand'],axis=1,inplace=True)

In [209]:
# additional information in pandas dataframe 

df_add = pd.DataFrame(additional_info)
df_add.head()

,Manufacturer,Item Weight,Net Quantity,Included Components,Best Sellers Rank,Item Dimensions LxWxH,ASIN,Customer Reviews,Date First Available,Packer,Importer,Generic Name
0,"Tesora-inspired by you, Address:Pratech Brands, 48, 4th floor, B wing, Todi Estate, Sun Mill Compound, Senapati Bapat Marg, Lower Parel, Mumbai, Maharashtra 400013 , Customer care email id : care@shoptesora.com Customer care no. : +91-9136091007",10 kg 500 g,1.00 units,"1U Air Oven, 1U Toasting Rack, 1U wire rack, 1U instruction Manual, 1U Tong, 1U Mesh basket, 1U Rottiserie fork, 1U Drip Tray","#25,217 in Home & Kitchen (See Top 100 in Home & Kitchen)\n#27 in Air Fryers",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wonderchef,18 kg,NaN,"1 Microwave, 1 Glass Tray, 1 Baking Tray","#796,641 in Home & Kitchen (See Top 100 in Home & Kitchen)\n#229 in Microwave Ovens",58 x 46 x 34 Centimeters,NaN,NaN,NaN,NaN,NaN,NaN
2,"Faber, FRANKE FABER INDIA PRIVATE LIMITED 1086/1/2, Sanaswadi, Nagar Road, TAL-Shirur, Pune, Maharashtra, 412208",37 kg,1 Count,NaN,"#157,436 in Home & Kitchen (See Top 100 in Home & Kitchen)\n#79 in Microwave Ovens",59.4 x 54.8 x 59.4 Centimeters,B0821ZJR98,3.7\n37 ratings\n3.7 out of 5 stars,26 November 2019,"FRANKE FABER INDIA PRIVATE LIMITED 1086/1/2, Sanaswadi, Nagar Road, TAL-Shirur, Pune, Maharashtra, 412208","FRANKE FABER INDIA PRIVATE LIMITED 1086/1/2, Sanaswadi, Nagar Road, TAL-Shirur, Pune, Maharashtra, 412208",Microwave Ovens
3,NaN,NaN,NaN,NaN,NaN,32.4 x 45.1 x 26 Centimeters,B008S792R4,4.2\n40 ratings\n4.2 out of 5 stars,27 July 2017,NaN,NaN,NaN
4,Essencedelight,NaN,1.00 count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
# Separating the home kitchen rank and the sub-category rank as separate columns

df_add['Home & Kitchen Rank'] = df_add['Best Sellers Rank'].str.split('\n').str[0].str.split('in Home & Kitchen').str[0].str.replace('#', '').str.replace(',', '').str.strip()
df_add['Microwave Ovens Rank'] = df_add['Best Sellers Rank'].str.split('\n').str[1].str.split('in Microwave Ovens').str[0].str.replace('#', '').str.replace(',', '').str.strip()

In [211]:
# Removing the reduntant columns

df_add = df_add.drop(['Customer Reviews','Best Sellers Rank','Manufacturer'],axis=1)
df_add.head()

,Item Weight,Net Quantity,Included Components,Item Dimensions LxWxH,ASIN,Date First Available,Packer,Importer,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
0,10 kg 500 g,1.00 units,"1U Air Oven, 1U Toasting Rack, 1U wire rack, 1U instruction Manual, 1U Tong, 1U Mesh basket, 1U Rottiserie fork, 1U Drip Tray",NaN,NaN,NaN,NaN,NaN,NaN,25217,27 in Air Fryers
1,18 kg,NaN,"1 Microwave, 1 Glass Tray, 1 Baking Tray",58 x 46 x 34 Centimeters,NaN,NaN,NaN,NaN,NaN,796641,229
2,37 kg,1 Count,NaN,59.4 x 54.8 x 59.4 Centimeters,B0821ZJR98,26 November 2019,"FRANKE FABER INDIA PRIVATE LIMITED 1086/1/2, Sanaswadi, Nagar Road, TAL-Shirur, Pune, Maharashtra, 412208","FRANKE FABER INDIA PRIVATE LIMITED 1086/1/2, Sanaswadi, Nagar Road, TAL-Shirur, Pune, Maharashtra, 412208",Microwave Ovens,157436,79
3,NaN,NaN,NaN,32.4 x 45.1 x 26 Centimeters,B008S792R4,27 July 2017,NaN,NaN,NaN,NaN,NaN
4,NaN,1.00 count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
df_add.drop(['Included Components'],axis=1,inplace=True)

In [213]:
# Concatenating all details in one dataframe

df = df2.join(dfp['Brand']).join(df_tech).join(df_add)
df.head()

,product_url,product_title,mrp,sales_price,discount_percent,star_rating,number_of_ratings,seller_name,product_details,technical_details,additional_information,Brand,Special Feature,Product Dimensions,Colour,Capacity,Material,Recommended Uses For Product,Output Wattage,Wattage,Voltage,Control Method,Has Nonstick Coating,Inner Material,Min Temperature Setting,Controller Type,Is Dishwasher Safe,Max Temperature Setting,Manufacturer,Country of Origin,Item model number,Model,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Fuel Type,Defrost System,Door Orientation,Included Components,Batteries Required,Model Name,Model Year,Mounting Hardware,Colour Screen,Audio Wattage,Batteries Included,GSM frequencies,Device interface - primary,Does it contain liquid?,Includes Rechargeable Battery,Is there a timer?,Includes remote,Item part number,Energy Consumption,Form Factor,Control Type,Door Style,Finish Type,Door Material Type,Volume Capacity Name,Energy Efficiency,Lighting,Certification,Control Console,Imported By,Number of Memory Sticks,Special features,Other display features,Color,Item Model Number,Item Part Number,Primary material,What is in the box?,Shelf Type,Whats in the box,Is Assembly Required,Assembly Type,Number of Pieces,Number of Shelves,Locking Mechanism,Weight,Item Height,Item Width,Door Material,Chamber Volume,Number of Heating Elements,Power Source,Item Weight,Net Quantity,Item Dimensions LxWxH,ASIN,Date First Available,Packer,Importer,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
0,https://aax-eu.amazon.in/x/c/RDJEAB3aDVDmxJ_vZ7DYMF0AAAGI_BfknQMAAAH2AQBvbm9fdHhuX2JpZDIgICBvbm9fdHhuX2ltcDEgICCzh4dz/https://www.amazon.in/Tesora-Replaces-Rotisserie-Circulation-Technology/dp/B09VGTZYTZ/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c%3Aamzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c&cv_ct_cx=microwave+oven&keywords=microwave+oven&pd_rd_i=B09VGTZYTZ&pd_rd_r=e5c00fcb-e30a-45c6-981c-a19f34df80fb&pd_rd_w=giVfb&pd_rd_wg=mcTd7&pf_rd_p=f94148a7-b034-4e58-9f1d-2cf8db3cd03c&pf_rd_r=GEJPEFFYQN55K6VBER81&qid=1687856604&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-113-06195d41-65bd-48f4-8e70-03d97b6094ec,"Tesora Digital Air Fryer Oven | Replaces OTG, Oven, Air Fryer, Toaster, Rotisserie, OTG | 360° Turbo Hot Air Circulation Technology & Dual Heating Element| 1700 Watts | Black.(14.5L) (14.5L)",18500,12999,30,3.9,117,tesora - inspired by you,"{'Special Feature': 'Temperature Control', 'Product Dimensions': '30D x 30W x 30H Centimeters', 'Colour': 'Black', 'Capacity': '14.5 litres', 'Material': 'Aluminium'}","{'Special Feature': 'Temperature Control', 'Product Dimensions': '30D x 30W x 30H Centimeters', 'Colour': 'Black', 'Capacity': '14.5 litres', 'Material': 'Aluminium', 'Recommended Uses For Product': 'Indoor Kitchen', 'Output Wattage': '1700 Watts', 'Item Weight': '10.5 Kilograms', 'Brand': 'Tesora - Inspired by you', 'Wattage': '1700 Watts', 'Voltage': '230 Volts', 'Control Method': 'Touch', 'Has Nonstick Coating': 'Yes', 'Inner Material': 'Stainless Steel', 'Min Temperature Setting': '50 Degrees Celsius', 'Controller Type': 'Button', 'Is Dishwasher Safe': 'Yes', 'Max Temperature Setting': '220 Degrees Celsius', 'Manufacturer': 'Tesora-inspired by you', 'Country of Origin': 'China', 'Item model number': '14.5L', 'ASIN': 'B09VGTZYTZ'}","{'Manufacturer': 'Tesora-inspired by you, Address:Pratech Brands, 48, 4th floor, B wing, Todi Estate, Sun Mill Compound, Senapati Bapat Marg, Lower Parel, Mumbai, Maharashtra 400013 , Customer care email id : care@shoptesora.com Customer care no. : +91-9136091007', 'Item Weight': '10 kg 500 g', 'Net Quantity': '1.00 units', 'Included Components': '1U Air Oven, 1U Toasting Rack, 1U wire rack, 1U instruction Manual, 1U Tong, 1U Mesh basket, 1U Rottiserie fork, 1U Drip Tray', 'Best Sellers Rank': '#25,217 in Home & Kitchen (See Top 100 in Home & Kitchen) #27 in Air Fryers'}",NaN,Temperature Control,30D x 30W x 30H Cen

In [214]:
df.shape

(210, 100)

##### <font color='purple'><I>Filtering out the products that are microwave oven accessories from actual microwave ovens :

In [216]:
# filtering the products with price less than 5000

df.loc[df['sales_price'] < 5000]

,product_url,product_title,mrp,sales_price,discount_percent,star_rating,number_of_ratings,seller_name,product_details,technical_details,additional_information,Brand,Special Feature,Product Dimensions,Colour,Capacity,Material,Recommended Uses For Product,Output Wattage,Wattage,Voltage,Control Method,Has Nonstick Coating,Inner Material,Min Temperature Setting,Controller Type,Is Dishwasher Safe,Max Temperature Setting,Manufacturer,Country of Origin,Item model number,Model,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Fuel Type,Defrost System,Door Orientation,Included Components,Batteries Required,Model Name,Model Year,Mounting Hardware,Colour Screen,Audio Wattage,Batteries Included,GSM frequencies,Device interface - primary,Does it contain liquid?,Includes Rechargeable Battery,Is there a timer?,Includes remote,Item part number,Energy Consumption,Form Factor,Control Type,Door Style,Finish Type,Door Material Type,Volume Capacity Name,Energy Efficiency,Lighting,Certification,Control Console,Imported By,Number of Memory Sticks,Special features,Other display features,Color,Item Model Number,Item Part Number,Primary material,What is in the box?,Shelf Type,Whats in the box,Is Assembly Required,Assembly Type,Number of Pieces,Number of Shelves,Locking Mechanism,Weight,Item Height,Item Width,Door Material,Chamber Volume,Number of Heating Elements,Power Source,Item Weight,Net Quantity,Item Dimensions LxWxH,ASIN,Date First Available,Packer,Importer,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
4,https://www.amazon.in/Essencedelight-Microwave-Dustproof-Protector-Decorative/dp/B0897SXY7X/ref=sr_1_197?keywords=microwave+oven&qid=1687856619&sr=8-197,"Essencedelight Microwave Oven Cover Dustproof Cotton Machine Protector Decorative Kitchen Appliance Cover with Side Storage Pockets,lattice",4199,2089,50,NaN,<NA>,Paras Creations G,{},"{'Manufacturer': 'Essencedelight', 'Country of Origin': 'China', 'Item part number': 'Essencedelight', 'ASIN': 'B0897SXY7X'}","{'Manufacturer': 'Essencedelight', 'Net Quantity': '1.00 count'}",NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Essencedelight,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Essencedelight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00 count,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,https://www.amazon.in/Polyester-3-Layered-Microwave-Suitable-Design/dp/B0C6GVKMQP/ref=sr_1_97?keywords=microwave+oven&qid=1687856601&sr=8-97,ANGEL SALES Polyester 3-Layered Microwave Oven Top Cover With 4 Utility Pockets Suitable for Upto 30 Liter (Design 2),599,199,67,NaN,<NA>,BABA ACHALNATH,{},"{'Country of Origin': 'India', 'Item model number': 'Design 2', 'ASIN': 'B0C6GVKMQP'}","{'Included Components': 'No', 'Best Sellers Rank': '#741,552 in Home & Kitchen (See Top 100 in Home & Kitchen) #218 in Microwave Ovens'}",NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,India,Design 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741552,218
83,https://www.amazon.in/OneAssist-Extended-Warranty-Microwave-30001-40000/dp/B0BZGMWPT5/ref=sr_1_146?keywords=microwave+oven&qid=1687856610&sr=8-146,OneAssist 2Years Extended Warranty for Microwave Oven Up to 9 Months Old Rs. (30001-40000) (Email Delivery Only),6758,3379,50,NaN,<NA>,One Assist,{},"{'Manufacturer': 'OneAssist', 'Country of Origin': 'India', 'ASIN': 'B0BZGMWPT5'}","{'Manufacturer': 'OneAssist, 08080333333', 'Item Dimensions LxWxH': '12 x 25 x 25 Millimeters', 'Net Quantity': '1.00 count'}",NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OneAssist,India,NaN,NaN,NaN,NaN,NaN

In [232]:
dfc = df.copy()

In [233]:
# dropping those rows

dfc = dfc.drop(dfc[dfc['sales_price'] < 5000].index)

In [234]:
# checking further for the products that contain 'cover' in their title

selected_rows = dfc[dfc['product_title'].str.contains('cover', case=False)]
selected_rows

,product_url,product_title,mrp,sales_price,discount_percent,star_rating,number_of_ratings,seller_name,product_details,technical_details,additional_information,Brand,Special Feature,Product Dimensions,Colour,Capacity,Material,Recommended Uses For Product,Output Wattage,Wattage,Voltage,Control Method,Has Nonstick Coating,Inner Material,Min Temperature Setting,Controller Type,Is Dishwasher Safe,Max Temperature Setting,Manufacturer,Country of Origin,Item model number,Model,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Fuel Type,Defrost System,Door Orientation,Included Components,Batteries Required,Model Name,Model Year,Mounting Hardware,Colour Screen,Audio Wattage,Batteries Included,GSM frequencies,Device interface - primary,Does it contain liquid?,Includes Rechargeable Battery,Is there a timer?,Includes remote,Item part number,Energy Consumption,Form Factor,Control Type,Door Style,Finish Type,Door Material Type,Volume Capacity Name,Energy Efficiency,Lighting,Certification,Control Console,Imported By,Number of Memory Sticks,Special features,Other display features,Color,Item Model Number,Item Part Number,Primary material,What is in the box?,Shelf Type,Whats in the box,Is Assembly Required,Assembly Type,Number of Pieces,Number of Shelves,Locking Mechanism,Weight,Item Height,Item Width,Door Material,Chamber Volume,Number of Heating Elements,Power Source,Item Weight,Net Quantity,Item Dimensions LxWxH,ASIN,Date First Available,Packer,Importer,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
79,https://www.amazon.in/ELECTROPRIME-Dustproof-Microwave-Convection-Multicolored/dp/B0836NX8H7/ref=sr_1_200?keywords=microwave+oven&qid=1687856619&sr=8-200,ELECTROPRIME Dustproof Cloth Cover Microwave Oven Cover Style Litre Convection Microwave Oven (Multicolored Wave Point),<NA>,<NA>,<NA>,NaN,<NA>,NaN,"{'Brand': 'ELECTROPRIME', 'Colour': 'Multicolour', 'Material': 'Others', 'Included Components': 'No', 'Model Name': 'DSD00935'}","{'Color': 'Multicolour', 'Item Model Number': 'RENHOSCLFC4599', 'Item Part Number': 'RENHOSCLFC4599', 'Primary material': 'Others', 'Capacity': 'Standard', 'What is in the box?': 'No', 'Manufacturer': 'ELECTROPRIME'}","{'ASIN': 'B0836NX8H7', 'Date First Available': '27 December 2019', 'Manufacturer': 'ELECTROPRIME'}",ELECTROPRIME,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ELECTROPRIME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multicolour,RENHOSCLFC4599,RENHOSCLFC4599,Others,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B0836NX8H7,27 December 2019,NaN,NaN,NaN,NaN,NaN
151,https://www.amazon.in/ELECTROPRIME-Microwave-Dustproof-Assorted-Colour/dp/B082YK3HLC/ref=sr_1_198?keywords=microwave+oven&qid=1687856619&sr=8-198,ELECTROPRIME Microwave Oven Cover with 2 Pouch Dustproof Cover (Assorted Colour and Design),<NA>,<NA>,<NA>,NaN,<NA>,NaN,"{'Brand': 'ELECTROPRIME', 'Colour': 'Multicolour', 'Material': 'Others', 'Included Components': 'No', 'Model Name': 'DKT004311784N'}","{'Color': 'Multicolour', 'Item Model Number': 'RENCLFC1718', 'Item Part Number': 'RENCLFC1718', 'Primary material': 'Others', 'Capacity': 'Standard', 'What is in the box?': 'No', 'Manufacturer': 'ELECTROPRIME'}","{'ASIN': 'B082YK3HLC', 'Date First Available': '20 December 2019', 'Manufacturer': 'ELECTROPRIME'}",ELECTROPRIME,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ELECTROPRIME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Multicolour,RENCLFC1718,RENCLFC1718,Others,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B082YK3HLC,20 December 2019,NaN,NaN,NaN,NaN,NaN


In [227]:
# dropping those rows

dfc = dfc.drop(selected_rows.index)

##### <font color='purple'><I>Final Dataframe :

In [235]:
dfc

,product_url,product_title,mrp,sales_price,discount_percent,star_rating,number_of_ratings,seller_name,product_details,technical_details,additional_information,Brand,Special Feature,Product Dimensions,Colour,Capacity,Material,Recommended Uses For Product,Output Wattage,Wattage,Voltage,Control Method,Has Nonstick Coating,Inner Material,Min Temperature Setting,Controller Type,Is Dishwasher Safe,Max Temperature Setting,Manufacturer,Country of Origin,Item model number,Model,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Fuel Type,Defrost System,Door Orientation,Included Components,Batteries Required,Model Name,Model Year,Mounting Hardware,Colour Screen,Audio Wattage,Batteries Included,GSM frequencies,Device interface - primary,Does it contain liquid?,Includes Rechargeable Battery,Is there a timer?,Includes remote,Item part number,Energy Consumption,Form Factor,Control Type,Door Style,Finish Type,Door Material Type,Volume Capacity Name,Energy Efficiency,Lighting,Certification,Control Console,Imported By,Number of Memory Sticks,Special features,Other display features,Color,Item Model Number,Item Part Number,Primary material,What is in the box?,Shelf Type,Whats in the box,Is Assembly Required,Assembly Type,Number of Pieces,Number of Shelves,Locking Mechanism,Weight,Item Height,Item Width,Door Material,Chamber Volume,Number of Heating Elements,Power Source,Item Weight,Net Quantity,Item Dimensions LxWxH,ASIN,Date First Available,Packer,Importer,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
0,https://aax-eu.amazon.in/x/c/RDJEAB3aDVDmxJ_vZ7DYMF0AAAGI_BfknQMAAAH2AQBvbm9fdHhuX2JpZDIgICBvbm9fdHhuX2ltcDEgICCzh4dz/https://www.amazon.in/Tesora-Replaces-Rotisserie-Circulation-Technology/dp/B09VGTZYTZ/ref=sxbs_sbv_search_btf?content-id=amzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c%3Aamzn1.sym.f94148a7-b034-4e58-9f1d-2cf8db3cd03c&cv_ct_cx=microwave+oven&keywords=microwave+oven&pd_rd_i=B09VGTZYTZ&pd_rd_r=e5c00fcb-e30a-45c6-981c-a19f34df80fb&pd_rd_w=giVfb&pd_rd_wg=mcTd7&pf_rd_p=f94148a7-b034-4e58-9f1d-2cf8db3cd03c&pf_rd_r=GEJPEFFYQN55K6VBER81&qid=1687856604&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sr=1-113-06195d41-65bd-48f4-8e70-03d97b6094ec,"Tesora Digital Air Fryer Oven | Replaces OTG, Oven, Air Fryer, Toaster, Rotisserie, OTG | 360° Turbo Hot Air Circulation Technology & Dual Heating Element| 1700 Watts | Black.(14.5L) (14.5L)",18500,12999,30,3.9,117,tesora - inspired by you,"{'Special Feature': 'Temperature Control', 'Product Dimensions': '30D x 30W x 30H Centimeters', 'Colour': 'Black', 'Capacity': '14.5 litres', 'Material': 'Aluminium'}","{'Special Feature': 'Temperature Control', 'Product Dimensions': '30D x 30W x 30H Centimeters', 'Colour': 'Black', 'Capacity': '14.5 litres', 'Material': 'Aluminium', 'Recommended Uses For Product': 'Indoor Kitchen', 'Output Wattage': '1700 Watts', 'Item Weight': '10.5 Kilograms', 'Brand': 'Tesora - Inspired by you', 'Wattage': '1700 Watts', 'Voltage': '230 Volts', 'Control Method': 'Touch', 'Has Nonstick Coating': 'Yes', 'Inner Material': 'Stainless Steel', 'Min Temperature Setting': '50 Degrees Celsius', 'Controller Type': 'Button', 'Is Dishwasher Safe': 'Yes', 'Max Temperature Setting': '220 Degrees Celsius', 'Manufacturer': 'Tesora-inspired by you', 'Country of Origin': 'China', 'Item model number': '14.5L', 'ASIN': 'B09VGTZYTZ'}","{'Manufacturer': 'Tesora-inspired by you, Address:Pratech Brands, 48, 4th floor, B wing, Todi Estate, Sun Mill Compound, Senapati Bapat Marg, Lower Parel, Mumbai, Maharashtra 400013 , Customer care email id : care@shoptesora.com Customer care no. : +91-9136091007', 'Item Weight': '10 kg 500 g', 'Net Quantity': '1.00 units', 'Included Components': '1U Air Oven, 1U Toasting Rack, 1U wire rack, 1U instruction Manual, 1U Tong, 1U Mesh basket, 1U Rottiserie fork, 1U Drip Tray', 'Best Sellers Rank': '#25,217 in Home & Kitchen (See Top 100 in Home & Kitchen) #27 in Air Fryers'}",NaN,Temperature Control,30D x 30W x 30H Cen

In [228]:
len(dfc)

199

In [229]:
dfc.shape

(199, 100)

In [230]:
dfc.to_csv('Amazon_microwave_oven.csv',index=False)